In [112]:
import tensorflow as tf

In [113]:
attributes = ["5_o_Clock_Shadow", "Bald", "Black_Hair", "Blond_Hair", "Brown_Hair",
              "Eyeglasses", "Goatee", "Gray_Hair", "Male", "Mustache", "Smiling", "No_Beard",
              "Wearing_Earrings", "Wearing_Hat"]

In [114]:
saved_model_path = "/home/ali/projects/face-describer/saved_model/my_model_tflite_compatible/"
model = tf.keras.models.load_model(saved_model_path)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('my_model.tflite', 'wb') as f:
    f.write(tflite_model)

2024-01-05 13:15:43.545836: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1280]
	 [[{{node inputs}}]]
2024-01-05 13:15:44.772948: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1280]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /tmp/tmpjseid9ui/assets


INFO:tensorflow:Assets written to: /tmp/tmpjseid9ui/assets
2024-01-05 13:16:01.587180: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-01-05 13:16:01.587294: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-01-05 13:16:01.587704: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpjseid9ui
2024-01-05 13:16:01.632919: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-01-05 13:16:01.632979: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpjseid9ui
2024-01-05 13:16:01.775754: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-01-05 13:16:02.489893: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmpjseid9ui
2024-01-05 13:16:02.764472: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

In [115]:
interpreter = tf.lite.Interpreter(model_path='my_model.tflite')
interpreter.allocate_tensors()

# Get input details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Load and preprocess image
image_path = '/home/ali/projects/face-describer/download.jpeg'
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(input_details[0]['shape'][1], input_details[0]['shape'][2]))
image_array = tf.keras.preprocessing.image.img_to_array(image)
image_array = tf.expand_dims(image_array, 0)  # Add batch dimension
image_array = image_array / 255.0  # Normalize to [0,1]

# Resize input tensor if necessary
interpreter.resize_tensor_input(input_details[0]['index'], (1, input_details[0]['shape'][1], input_details[0]['shape'][2], 3))

interpreter.set_tensor(input_details[0]['index'], image_array)
interpreter.invoke()



In [116]:
sorted_output_details = sorted(output_details, key=lambda x: x['index'])

output_data = [interpreter.get_tensor(sorted_output_details[i]['index']) for i in range(len(output_details))]
print("Inference result:", output_data)

Inference result: [array([[0.00010051]], dtype=float32), array([[6.0689113e-06]], dtype=float32), array([[5.9010704e-06]], dtype=float32), array([[0.98781663]], dtype=float32), array([[0.00253838]], dtype=float32), array([[0.9722426]], dtype=float32), array([[3.3731787e-05]], dtype=float32), array([[0.00426359]], dtype=float32), array([[0.00036029]], dtype=float32), array([[1.43698635e-05]], dtype=float32), array([[0.9999305]], dtype=float32), array([[0.04218395]], dtype=float32), array([[0.2303866]], dtype=float32), array([[0.0002588]], dtype=float32)]


In [117]:
for i in range (len(output_data)):
    print (attributes[i],"        " ,  output_data[i][0][0] > 0.5, output_data[i][0][0])

5_o_Clock_Shadow          False 0.00010050917
Bald          False 6.0689113e-06
Black_Hair          False 5.9010704e-06
Blond_Hair          True 0.98781663
Brown_Hair          False 0.0025383774
Eyeglasses          True 0.9722426
Goatee          False 3.3731787e-05
Gray_Hair          False 0.0042635873
Male          False 0.00036028773
Mustache          False 1.43698635e-05
Smiling          True 0.9999305
No_Beard          False 0.042183954
Wearing_Earrings          False 0.2303866
Wearing_Hat          False 0.00025879798
